In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import contextlib
import json
import os
import string
import tarfile
import tempfile
import time
import zipfile
from pathlib import Path

import boto3
import mysql

import reseval

In [ ]:
# Create a globally unique name to prevent collision
unique = (
    reseval.random.string(1, string.ascii_lowercase) +
    reseval.random.string(23))

# Save unique name
credentials_file = (
    reseval.EVALUATION_DIRECTORY /
    unique /
    'credentials' /
    'unique.json')
credentials_file.parent.mkdir(exist_ok=True, parents=True)
with open(credentials_file, 'w') as file:
    json.dump({'unique': unique}, file)

## Create

### Storage

In [ ]:
# Connect to S3
client = reseval.storage.aws.connect()

# Create bucket
client.create_bucket(Bucket=unique, ObjectOwnership='ObjectWriter')

# Set bucket policy to public read-only
client.put_public_access_block(
    Bucket=unique,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': False,
        'IgnorePublicAcls': False,
        'BlockPublicPolicy': False,
        'RestrictPublicBuckets': False
    })
client.put_bucket_acl(ACL='public-read', Bucket=unique)

# Load CORS policy as JSON
with open(reseval.ASSETS_DIR / 'cors.json') as file:
    cors = {'CORSRules': json.load(file)}

# Set bucket CORS to allow reads
client.put_bucket_cors(Bucket=unique, CORSConfiguration=cors)

### Database

In [ ]:
@contextlib.contextmanager
def connect():
    """Connect to a MySQL database"""
    try:

        # Connect to the database
        connection = mysql.connector.connect(
            database=os.environ['RDS_DB_NAME'],
            host=os.environ['RDS_HOSTNAME'],
            user=os.environ['RDS_USERNAME'],
            password=os.environ['RDS_PASSWORD'])

        # Create cursor to execute commands
        cursor = connection.cursor()

        # Execute user code
        yield connection, cursor

    finally:

        # Close database connection
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals():
            connection.close()
            
# Connect to AWS
client = reseval.database.aws.connect()

# Create database
response = client.create_db_instance(
    DBName=unique,
    DBInstanceIdentifier=unique,
    DBInstanceClass='db.t2.micro',
    AllocatedStorage=64,
    Engine='mysql',
    MasterUsername='root',
    MasterUserPassword='password',
    PubliclyAccessible=True)['DBInstance']

print(response)

# Get credentials
while response['DBInstanceStatus'] == 'creating':
    time.sleep(5)
    response = client.describe_db_instances(
        DBInstanceIdentifier=unique
    )['DBInstances'][0]
credentials = {
    'RDS_HOSTNAME': response['Endpoint']['Address'],
    'RDS_PORT': '3306',
    'RDS_DB_NAME': unique,
    'RDS_USERNAME': 'root',
    'RDS_PASSWORD': 'password'}

# Save environment variables
for key, value in credentials.items():
    os.environ[key] = value

# Connect to MySQL database
with connect() as (_, cursor):
    
    # Create database tables
    for table, command in zip(reseval.database.TABLES, reseval.database.SCHEMA):

        # Add prescreen questions from configuration file
        if table == 'participants':

            # Update MySQL command with prescreen questions
            command = command.format('')

        # Response type is unique to the test
        if table == 'responses':
            continue

        # Communicate with database
        cursor.execute(command)

### Server

In [ ]:
# Connect to AWS
client = reseval.server.aws.connect()

# Create elastic beanstalk application
response = client.create_application(ApplicationName=unique)

print(response)

# Create elastic beanstalk environment
response = client.create_environment(
    ApplicationName=unique,
    EnvironmentName=unique,
    SolutionStackName='64bit Amazon Linux 2 v5.8.3 running Node.js 18',
    OptionSettings=[
        {
            'Namespace': 'aws:elasticbeanstalk:application:environment',
            'OptionName': key,
            'Value': value
        } for key, value in credentials.items()
    ] + [{
        'Namespace': 'aws:autoscaling:launchconfiguration',
        'OptionName': 'IamInstanceProfile',
        'Value': 'aws-elasticbeanstalk-ec2-role'
    }])

print(response)

# Maybe install client
client_directory = reseval.CACHE / 'client'
if not (client_directory / 'node_modules').exists():
    with reseval.chdir(client_directory):
        reseval.npm.install().wait()

# Build client
with reseval.chdir(reseval.CACHE / 'client'):
    reseval.npm.build().wait()

In [ ]:
# Create a tarball of all files needed by the Heroku server
with tempfile.TemporaryDirectory() as directory:
    directory = Path(directory)

    # TODO - zipfile organization
    paths = [
        'client/build',
        'server',
        'package-lock.json',
        'package.json',
        'Procfile',
        'server.ts',
        'tsconfig.json']
    with zipfile.ZipFile(directory / 'reseval.zip', 'w') as bundle:
        for path in paths:
            bundle.write(reseval.CACHE / file, "dir\\test.py", zipfile.ZIP_DEFLATED)
    
    # Upload tarball
    reseval.storage.aws.upload(unique, directory / 'reseval.zip')

# Update application
response = client.create_application_version(
    ApplicationName=unique,
    VersionLabel='0.0.1',
    SourceBundle={
        'S3Bucket': unique,
        'S3Key': 'reseval.zip'
    }
)

# Wait for build to finish
while response['Status'] in ['Building', 'Processing', 'Unprocessed']:
    time.sleep(5)
    response = client.describe_application_versions(
        ApplicationName=unique,
        VersionLabels=['0.0.1']
    )['ApplicationVersions'][0]['ApplicationVersion']

# Application URL
region = response['ApplicationVersionArn'].split(':')[3]
url = f'{unique}-{region}.elasticbeanstalk.com'
print(url)

## Delete

### Server

In [ ]:
# Get S3 bucket name
arn = client.describe_application_versions(
        ApplicationName=unique,
        VersionLabels=['0.0.1']
    )['ApplicationVersions'][0]['ApplicationVersionARN']
region = arn.split(':')[3]
bucket = arn.split(':')[4]

# Connect to AWS
client = reseval.server.aws.connect()

# Delete environment
client.terminate_environment(
    EnvironmentName=unique,
    TerminateResources=True,
    ForceTerminate=True)

# Connect to S3 bucket
bucket = boto3.Session(
    aws_access_key_id=os.environ['AWSAccessKeyId'],
    aws_secret_access_key=os.environ['AWSSecretKey']
).resource('s3').Bucket(f'elasticbeanstalk-{region}-{bucket}')

# Delete contents
bucket.objects.all().delete()

# Delete bucket
bucket.delete()

### Database

In [ ]:
# Connect to AWS
client = reseval.database.aws.connect()

# Delete database
client.delete_db_instance(
    DBInstanceIdentifier=unique,
    SkipFinalSnapshot=True,
    DeleteAutomatedBackups=True)

### Storage

In [ ]:
# Connect to S3 bucket
bucket = boto3.Session(
    aws_access_key_id=os.environ['AWSAccessKeyId'],
    aws_secret_access_key=os.environ['AWSSecretKey']
).resource('s3').Bucket(unique)

# Delete contents
bucket.objects.all().delete()

# Delete bucket
bucket.delete()